In [1]:
import cv2
import numpy as np
import os
import time
from matplotlib import pyplot as plt
import mediapipe as mp
import pyttsx3
engine=pyttsx3.init('sapi5')
voices=engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)
def speak(text):
    engine.say(text)
    engine.runAndWait()

In [2]:
#KEYPOINTS USING MP 

'''Creating two variables, one for mediapipe holistic and one 
for drawing utilities, holistic is gonna be downloading that model 
and leveraging that model and drawing utility is for the ease of drawing
keypoints on our face '''

mp_holistic = mp.solutions.holistic         
mp_drawing = mp.solutions.drawing_utils 

def mediapipe_detection(image, model):
    '''We are gonna grab the image and convert it from bgr to rgb and then set it to unwritieable so 
    this saves a little bit of memory'''
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False 
    results = model.process(image) 
    '''coverting it back to writeable and then changing color from 
    rgb to bgr '''
    image.flags.writeable = True 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results


def draw_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
'''cap=cv2.VideoCapture(1)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #read input
        ret, frame = cap.read() 
        #make detections
        image, results = mediapipe_detection(frame,holistic) 
        print(results)
        #draw landmarks
        draw_landmarks(image,results)
        #generate output
        cv2.imshow('opencv feed',image) 
        #break
        if cv2.waitKey(10) & 0xFF == ord('q'): 
            break
    cap.release()
    cv2.destroyAllWindows'''

"cap=cv2.VideoCapture(1)\n#set mediapipe model\nwith mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:\n    while cap.isOpened():\n        #read input\n        ret, frame = cap.read() \n        #make detections\n        image, results = mediapipe_detection(frame,holistic) \n        print(results)\n        #draw landmarks\n        draw_landmarks(image,results)\n        #generate output\n        cv2.imshow('opencv feed',image) \n        #break\n        if cv2.waitKey(10) & 0xFF == ord('q'): \n            break\n    cap.release()\n    cv2.destroyAllWindows"

In [3]:
#EXTRACT KEYPOINT VALUES
def extract_keypoints(results):
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

In [4]:
#SETUP FOLDERS FOR COLLECTION

DATA_PATH = os.path.join('MP_Data') # Path for exported data, numpy arrays

actions = np.array(['Netflix', 'Spotify', 'Gesture not found']) # Actions that we try to detect

no_sequences = 30 # Thirty videos worth of data

sequence_length = 30 # Videos are going to be 30 frames in length

In [5]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [6]:
#COLLECT KEYPOINT VALUES FOR TRAINING AND TESTING 
cap=cv2.VideoCapture(1)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    #Loop through actions
    for action in actions:
        #Loop through sequences
        for sequence in range(no_sequences):
            #Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                
                #read input
                ret, frame = cap.read() 
                
                #make detections
                image, results = mediapipe_detection(frame,holistic) 
                print(results)
                
                #draw landmarks
                draw_landmarks(image,results)
                
                #new apply wait logic
                if frame_num==0:
                    cv2.putText(image,'STARTING COLLECTION',(120,200),
                                cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image,'Collecting frames for {} video number {}'.format(action,sequence),(15,12),
                                cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image,'Collecting frames for {} video number {}'.format(action,sequence),(15,12),
                                cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    
                #new export keypoints
                keypoints=extract_keypoints(results)
                npy_path=os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints) 
                
                #show to screen
                cv2.imshow('opencv feed',image) 
                #break
                if cv2.waitKey(10) & 0xFF == ord('q'): 
                    break
cap.release()
cv2.destroyAllWindows

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<function destroyAllWindows>

In [7]:
#PREPROCESS DATA AND CREATE LABELS AND FEATURES

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

label_map= { label:num for num, label in enumerate(actions) }

sequences, labels=[],[]
for action in actions:
    for sequence in range(no_sequences): 
        window=[] 
        for frame_num in range(sequence_length):
            res=np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action]) 


X=np.array(sequences)
y=to_categorical(labels).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [8]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [9]:
#BUILD AND TRAIN LSTM NEURAL NETWORK
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard


log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 5s 93ms/step - loss: 1.5405 - categorical_accuracy: 0.3647
Epoch 2/2000
3/3 [==============================] - 0s 94ms/step - loss: 1.2292 - categorical_accuracy: 0.3412
Epoch 3/2000
3/3 [==============================] - 0s 97ms/step - loss: 1.1681 - categorical_accuracy: 0.3647
Epoch 4/2000
3/3 [==============================] - 0s 94ms/step - loss: 2.2079 - categorical_accuracy: 0.2941
Epoch 5/2000
3/3 [==============================] - 0s 94ms/step - loss: 3.6017 - categorical_accuracy: 0.3059
Epoch 6/2000
3/3 [==============================] - 0s 95ms/step - loss: 1.2434 - categorical_accuracy: 0.3882
Epoch 7/2000
3/3 [==============================] - 0s 98ms/step - loss: 1.2943 - categorical_accuracy: 0.4118
Epoch 8/2000
3/3 [==============================] - 0s 100ms/step - loss: 1.0347 - categorical_accuracy: 0.4353
Epoch 9/2000
3/3 [==============================] - 0s 93ms/step - loss: 0.9103 - categorical_accuracy: 0.4941


In [10]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

In [11]:
# MAKE SIGN LANGUAGE PREDICTIONS
res = model.predict(X_test)

1/1 [==============================] - 0s 351ms/step


In [12]:
actions[np.argmax(res[4])]

'Spotify'

In [13]:
actions[np.argmax(y_test[4])]

'Spotify'

In [14]:
model.save('action.h5')

In [20]:
model.load_weights('action.h5')

In [15]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [16]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 28ms/step


In [17]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [18]:
yhat

[2, 0, 1, 1, 1]

In [19]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 1],
        [0, 0]],

       [[1, 0],
        [1, 3]],

       [[4, 0],
        [0, 1]]], dtype=int64)

In [20]:
accuracy_score(ytrue,yhat)

0.8

In [21]:
from scipy import stats

In [22]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [23]:
import os
import webbrowser

In [24]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.7
engine.setProperty('voice', voices[1].id)
cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                            if actions[np.argmax(res)]=="Netflix":
                                speak('Netflix - open now !')
                                webbrowser.open_new_tab('https://www.netflix.com/browse ')
                            elif actions[np.argmax(res)]=="Spotify":
                                speak('Spotify - open now !')
                                webbrowser.open_new_tab('https://open.spotify.com/')
                            else:
                                speak(actions[np.argmax(res)])    
                        
                    else:
                        sentence.append(actions[np.argmax(res)])
                        if actions[np.argmax(res)]=="Netflix":
                            speak('Netflix - open now !')
                            webbrowser.open_new_tab('https://www.netflix.com/browse ')
                        elif actions[np.argmax(res)]=="Spotify":
                            speak('Spotify - open now !')
                            webbrowser.open_new_tab('https://open.spotify.com/')
                        else:
                            speak(actions[np.argmax(res)])    
                        
            
            

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)

            
        
        

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image) 

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti